In [5]:
import math
import networkx as nx
import osmnx as ox
from geopy.distance import great_circle

# Define locations
locations = {
    "Marienplatz": "Marienplatz, Munich, Germany",
    "Olympiapark": "Olympiapark, Munich, Germany",
    "HBF": "Hauptbahnhof, Munich, Germany",
    "BMW": "BMW Welt, Munich, Germany",
    "TUM City Center": "Arcisstraße 21, 80333 Munich, Germany",
    "Münchener Freiheit": "Münchener Freiheit, Munich, Germany"
}

# Geocode locations
coords = {name: ox.geocode(loc) for name, loc in locations.items()}

# Create a network graph for Munich
G = ox.graph_from_address("Munich, Germany", network_type='drive', dist=4000)

# Find the nearest nodes to the locations
nearest_nodes = {name: ox.distance.nearest_nodes(G, point[1], point[0]) for name, point in coords.items()}

# Function to calculate the centroid of coordinates
def calculate_centroid(coords):
    latitudes = [coord[0] for coord in coords.values()]
    longitudes = [coord[1] for coord in coords.values()]
    return sum(latitudes) / len(latitudes), sum(longitudes) / len(longitudes)

# Calculate the centroid of the nodes
centroid = calculate_centroid(coords)

# Function to calculate distance between two points
def calculate_distance(point1, point2):
    return great_circle(point1, point2).kilometers

# Find the maximum distance from the centroid to any node
max_distance_to_centroid = max(calculate_distance(centroid, point) for point in coords.values())

# Calculate the area of the circle
area_of_circle = math.pi * (max_distance_to_centroid ** 2)

# Munich's population density (people per square km)
population_density = 4900  # Adjust as needed

# Calculate C based on the area and population density
C = area_of_circle * population_density

# Custom function to estimate trips
def estimate_trips(distance, C, alpha):
    return C * math.exp(-alpha * distance)

# Constants for the model
alpha = 0.1  # Decay constant, adjust based on scenario

# Define the routes as a dictionary of O-D pairs
routes = {
    "Marienplatz to Olympiapark": nx.shortest_path_length(G, nearest_nodes["Marienplatz"], nearest_nodes["Olympiapark"], weight='length'),
    "Olympiapark to HBF": nx.shortest_path_length(G, nearest_nodes["Olympiapark"], nearest_nodes["HBF"], weight='length'),
    "HBF to BMW": nx.shortest_path_length(G, nearest_nodes["HBF"], nearest_nodes["BMW"], weight='length'),
    "BMW to TUM City Center": nx.shortest_path_length(G, nearest_nodes["BMW"], nearest_nodes["TUM City Center"], weight='length'),
    "TUM City Center to Münchener Freiheit": nx.shortest_path_length(G, nearest_nodes["TUM City Center"], nearest_nodes["Münchener Freiheit"], weight='length'),
    "Münchener Freiheit to Olympiapark": nx.shortest_path_length(G, nearest_nodes["Münchener Freiheit"], nearest_nodes["Olympiapark"], weight='length'),
}

# Calculate estimated trips for each route
estimated_trips = {}
for route in routes.keys():
    distance = routes[route]/1000  # Distance for the route in km
    trips = estimate_trips(distance, C, alpha)
    estimated_trips[route] = trips

# Assuming 220 working days a year
annual_trips = {route: trips * 260 for route, trips in estimated_trips.items()}

# Print the estimated annual trips for each route
for route, trips in annual_trips.items():
    print(f"{route}: {trips} trips/year")

ImportError: scikit-learn must be installed to search an unprojected graph